In [2]:
import pandas as pd
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os

In [ ]:
# --- Configuración de OpenAI ---
# Establece tu API Key de OpenAI aquí
# Es MÁS SEGURO configurarla como una variable de entorno:
# os.environ["OPENAI_API_KEY"] = "TU_API_KEY_AQUI"
# Si la configuras como variable de entorno, puedes omitir la línea de abajo.
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')  # ¡REEMPLAZA ESTO CON TU CLAVE REAL!

# Inicializa el modelo de embeddings y el LLM
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
llm = ChatOpenAI(model="gpt-4o", temperature=0.1) # gpt-4o es potente y rápido, puedes probar con gpt-3.5-turbo también

In [4]:
# --- Paso 1: Cargar y Procesar el Archivo CSV ---
def load_and_process_csv(file_path):
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no se encontró.")
        return None

    # Convertir cada fila del DataFrame en un 'Document' de LangChain
    # Cada documento contendrá la información de una fila como su contenido.
    # Puedes ajustar qué columnas incluir en la representación del documento.
    documents = []
    for index, row in df.iterrows():
        # Aquí puedes personalizar cómo quieres que se represente cada fila como texto.
        # Por ejemplo, podemos unir todos los valores de la fila en una sola cadena.
        content = ", ".join([f"{col}: {row[col]}" for col in df.columns])
        # También puedes agregar metadatos si lo necesitas, por ejemplo, el índice de la fila
        metadata = {"row_index": index}
        documents.append(Document(page_content=content, metadata=metadata))
    
    print(f"Se cargaron {len(documents)} documentos del archivo CSV.")
    return documents

In [5]:
# --- Paso 2: Crear la Base de Datos Vectorial ---
def create_vector_db(documents, embeddings_model):
    print("Creando la base de datos vectorial... Esto puede tomar un momento.")
    # Usamos FAISS para una base de datos vectorial en memoria, sencilla para empezar.
    vector_db = FAISS.from_documents(documents, embeddings_model)
    print("Base de datos vectorial creada exitosamente.")
    return vector_db

In [6]:
# --- Paso 3: Crear el Agente de Consulta (RetrievalQA Chain) ---
def create_agent(vector_db, llm_model):
    # Un "retriever" es lo que buscará documentos relevantes en tu base de datos vectorial.
    retriever = vector_db.as_retriever()
    
    # RetrievalQA es una cadena que combina la recuperación de documentos con un LLM para responder preguntas.
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm_model, 
        chain_type="stuff", # 'stuff' simplemente toma todos los documentos recuperados y los 'mete' en el prompt del LLM
        retriever=retriever, 
        return_source_documents=True # Útil para ver qué partes del CSV se usaron para la respuesta
    )
    print("Agente de consulta configurado.")
    return qa_chain

In [7]:
### **Función Principal Modificada**

def main():
    # Define la ruta a tu archivo CSV existente.
    # Asegúrate de que 'coffee_final.csv' esté en la misma carpeta que tu script,
    # o proporciona la ruta completa al archivo.
    csv_file_path = "coffee_final.csv" 

    # Simplemente llamamos a la función para cargar y procesar el archivo existente.
    # ¡Las líneas para crear sample_data han sido eliminadas!
    documents = load_and_process_csv(csv_file_path)
    if not documents:
        # Si no se pudieron cargar los documentos (ej. el archivo no existe), el programa termina.
        return

    vector_db = create_vector_db(documents, embeddings)
    qa_agent = create_agent(vector_db, llm)

    print("\n¡Listo para hacer consultas! Escribe 'salir' para terminar.")
    while True:
        query = input("\nTu pregunta: ")
        if query.lower() == 'salir':
            print("¡Hasta luego!")
            break
        
        try:
            result = qa_agent.invoke({"query": query})
            print("\n--- Respuesta del Agente ---")
            print(result["result"])
            print("\n--- Documentos Fuente Utilizados ---")
            for doc in result["source_documents"]:
                print(f"- {doc.page_content}")
            print("-----------------------------\n")
        except Exception as e:
            print(f"Ocurrió un error al procesar la consulta: {e}")

In [8]:
if __name__ == "__main__":
    main()

Se cargaron 1430 documentos del archivo CSV.
Creando la base de datos vectorial... Esto puede tomar un momento.
Base de datos vectorial creada exitosamente.
Agente de consulta configurado.

¡Listo para hacer consultas! Escribe 'salir' para terminar.

--- Respuesta del Agente ---
La venta máxima de café en Colombia, según los datos proporcionados, fue de 99,927,060 en el año 2015.

--- Documentos Fuente Utilizados ---
- Fecha: 2001-01-01, Country: Colombia, Ventas: 76920000, Coffee type: Arabica
- Fecha: 2005-01-01, Country: Colombia, Ventas: 73380000, Coffee type: Arabica
- Fecha: 2015-01-01, Country: Colombia, Ventas: 99927060, Coffee type: Arabica
- Fecha: 1995-01-01, Country: Colombia, Ventas: 82500000, Coffee type: Arabica
-----------------------------


--- Respuesta del Agente ---
Lo siento, no tengo información sobre las ventas de café de Colombia entre los años 1992 y 1994. Solo tengo datos a partir de 1995, cuando las ventas fueron de 82,500,000.

--- Documentos Fuente Utiliza

In [ ]:
#!jupyter nbconvert --to script NB_agente_consulta_coffe.ipynb